# Matching Short Fingerprints with Database of Fingerprints

In [1]:
import librosa
import numpy as np
import os

# Define folders
songs_folder = "songs_mp3"  # Full song database
fingerprints_folder = "fingerprints"  # Precomputed fingerprints

def generate_snippet_fingerprint(file_path, start_time=10, duration=5):
    """Extracts a fingerprint from a short audio snippet"""
    try:
        # Load only a small snippet of the audio file
        y, sr = librosa.load(file_path, sr=44100, offset=start_time, duration=duration)
        
        # Compute spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        
        # Get peaks (top 5% of intensities)
        peaks = np.argwhere(D > np.percentile(D, 95))
        
        return peaks  # Return the fingerprint

    except Exception as e:
        print(f"⚠️ Error processing snippet: {e}")
        return None

def find_closest_match(snippet_fingerprint):
    """Find the song with the closest matching fingerprint"""
    best_match = None
    best_score = float('inf')  # Lower score means better match

    for fingerprint_file in os.listdir(fingerprints_folder):
        if fingerprint_file.endswith(".npy"):
            song_fingerprint = np.load(os.path.join(fingerprints_folder, fingerprint_file))
            
            # Compare snippet fingerprint to full song fingerprint
            difference = len(set(map(tuple, snippet_fingerprint)) - set(map(tuple, song_fingerprint)))

            if difference < best_score:  # Lower difference means better match
                best_score = difference
                best_match = fingerprint_file.replace(".npy", "")

    return best_match if best_match else "No match found"

# Test with a snippet from a song
test_song = os.path.join(songs_folder, "Michael Jackson - Billie Jean (Official Video).mp3")  # Change this to an actual song in your database
snippet_fingerprint = generate_snippet_fingerprint(test_song)

if snippet_fingerprint is not None:
    match = find_closest_match(snippet_fingerprint)
    print(f"Closest match: {match}")

🔍 Closest match: Michael Jackson - Billie Jean (Official Video)
